In [ ]:
import sys
import time
import pandas as pd
from langchain_ollama import OllamaEmbeddings
from ollama import Client
import json
import csv
import ast

In [11]:
CONFLICT_PROMPT_ONOS = """You are tasked with determining if two ONOS flow configuration JSONs directly conflict with each other. **You MUST base your decision SOLELY on the JSON content provided.** Do **NOT** infer intent, use semantic reasoning, or make assumptions beyond the given JSON structure.

---

### **Rules for Conflict Detection (STRICT and LITERAL COMPARISON):**

A **direct conflict** exists **ONLY IF** all the following conditions are met:

#### **1. Matching Traffic Characteristics (Exact or Overlapping Match Criteria)**
Both flows must match **overlapping traffic characteristics** for all the specified fields, meaning they could apply to the **exact same packets**. The following fields are checked:

**EtherType (`ETH_TYPE`)**  
   - Must be present in **both** flows and have **identical** values (e.g., `"0x800"` for IPv4, `"0x86DD"` for IPv6).
   - If missing in either flow, there is **no match**.

**Source and Destination IP (`IPV4_SRC` and `IPV4_DST`)**  
   - If both flows **specify** a source or destination, they must overlap (e.g., `"10.0.0.1/32"` and `"10.0.0.0/24"` overlap).
   - If **either** flow **omits** source or destination, it applies to **all** sources or destinations respectively.

**Protocol (`IP_PROTO`)**  
   - Must be present in **both** flows and have **identical** values (e.g., `6` for TCP, `17` for UDP).
   - If missing in either flow, there is **no match**.

**Transport Layer Ports (`TCP_SRC`, `TCP_DST`, `UDP_SRC`, `UDP_DST`)**  
   - Must be **identical** in both flows if present.
   - If one flow includes a port filter and the other does not, there is **no match**.

**Incoming Port (`IN_PORT`)**  
   - If both flows specify `IN_PORT`, they must be identical.
   - If missing in either flow, there is **no match**.

**Wildcard Matching (Implicit Behavior for Missing Fields)**  
   - If a flow **specifies a field** (e.g., `IPV4_SRC`), it applies **only** to that source.
   - If a flow **does not specify a field** (e.g., `IPV4_SRC`), it applies to **all sources**.
   - General vs. specific distinctions are not classified, but missing fields imply generality (i.e., match all). Conflicts are determined based on whether the two match sets could overlap.
---

#### **2. Contradictory Actions**
Flows that **match the same traffic** only conflict if their actions contradict in one of the following ways:

**Different Output Ports (`OUTPUT` instruction)**  
   - If one flow **forwards** traffic to port `X` and another to port `Y`, it is a **conflict**.

**One Flow Drops, One Flow Forwards**  
   - If one flow **omits `"treatment"`** (implying a drop) and another **forwards traffic**, it is a **conflict**.

**Different Queue Assignments (`QUEUE` instruction)**  
   - If the flows **assign different `queueId` values**, it is a **conflict**.

**Note on Additional Match Fields:**  
   - If one flow includes additional match fields, a conflict may still exist if the effective match space overlaps. Extra fields do not automatically prevent a conflict.

---

#### **3. Priority is Irrelevant for Conflict Detection**
- `priority` **does not** determine a conflict.
- Even if one rule **overrides** another, they **do not conflict** unless their actions contradict.

---

### **Input Format**
You will be provided with **two ONOS JSON flow rules** in the following format:

**Flow 1:**
```json
<JSON for Flow 1>

**Flow 2:**
<JSON for Flow 2>

### **Expected Output Format**
Respond strictly in valid JSON format, using the schema below:

{
    "conflict_status": <integer>,
    "conflict_explanation": "<conflict explanation, if any>"
}

Field Descriptions:
   - conflict_status should be 1 if a direct conflict exists, 0 otherwise.
   - conflict_explanation → A brief explanation if a conflict exists, otherwise an empty string "".

NO EXTRA TEXT, COMMENTS, OR EXPLANATIONS OUTSIDE JSON.
DO NOT return 1 unless you are certain of a direct conflict.
Follow strict field-by-field comparison rules—NO inference beyond given data."""

In [ ]:
my_models = [
"marco-o1",
"mistral",
"mistral-nemo",
"deepseek-coder",
"starcoder", 
"codegemma",
"starcoder2",
"openchat",
"phi3",
"dolphin-mistral",
"wizardlm2",
"phi",
"yi",
"zephyr",
"command-r",
"llava-llama3",
"codestral",
"codellama:34b",
"codellama",
"llama2",
"llama3",
"llama3.1",
"llama3.2",
"qwen",
"qwen2",
"qwen2.5",
"gemma2:27b",
"huihui_ai/qwq-abliterated",
"huihui_ai/qwq-fusion",
"qwq",
"llama3.3",
"llama2:70b",
"codellama:70b"
]

default_model = "llama2"

ollama_embedding_url = "http://10.23.7.63:11434"
ollama_server_url = "http://10.23.7.63:11435"  

ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

In [13]:
def run_LLM_conflict(flow_json_1, flow_json_2, model):
    
    system_prompt = CONFLICT_PROMPT_ONOS
    count = 0
    while True:
        count+=1
        try:
            time.sleep(0.1)

            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                stream=False,
                system=system_prompt,
                prompt=f"Flow 1:\n{json.dumps(flow_json_1, indent=2)}\n\nFlow 2:\n{json.dumps(flow_json_2, indent=2)}",
                format='json'
            )

            output = response['response'].strip()

            response_json = json.loads(output)

            if 'conflict_status' not in response_json:
                #print("\nWarning: 'conflict_status' key is missing in the response.\n")
                break
            else:
                valid_conflict_response = True
                conflict_status = response_json.get('conflict_status', 0)
                # Ensure conflict_status is an integer
                if isinstance(conflict_status, str):
                    conflict_status = int(conflict_status)

                return valid_conflict_response, conflict_status, response_json['conflict_explanation']             

        except Exception as e:
            print("Exception found: ", e)
            sys.stdout.flush()
            if(count<15):
                continue
            else:
                print("\n",model, " failed to produce valid JSON for conflict info after 15 tries. Going to next model\n")
                break               
    
    return False, None, None

In [14]:
def conflict(flow_json_1, flow_json_2):
    
    valid_conflict_response, conflict_status, conflict_details = run_LLM_conflict(flow_json_1, flow_json_2)

    if (valid_conflict_response == False):
        return 2, None  #2 means here that LLM could not generate valid JSON for conflict reporting
    elif (conflict_status == 1):
        return conflict_status, conflict_details
        
    return 0, None

In [15]:
def safe_load_flow_json(flow_str, index_number):
    try:
        return ast.literal_eval(flow_str)
    except Exception as e:
        print(f"[Index {index_number}] Error parsing flow JSON from excel using ast.literal_eval: {e}")
        return None


In [ ]:
df = pd.read_excel("FlowConflict-ONOS.xlsx")

# ---- Model-level summary collection ----
summary_rows = []

for model in my_models:
    TP = FP = TN = FN = 0
    times = []

    for idx, row in df.iterrows():

        index_number = row["Sl."]
        flow_rule_1_json = safe_load_flow_json(row["ONOS Flow Rule 1"], index_number)
        flow_rule_2_json = safe_load_flow_json(row["ONOS Flow Rule 2"], index_number)

        ground_truth_conflict = row["Conflicting"].strip().lower()  # 'yes' or 'no'
        gt_conflict = (ground_truth_conflict == 'yes')

        if (flow_rule_1_json is None) or (flow_rule_2_json is None):
            continue

        start_conflict_time = time.time()
        valid_conflict_response, conflict_status, conflict_details = run_LLM_conflict(
            flow_rule_1_json, flow_rule_2_json, model=model)
        conflict_duration = time.time() - start_conflict_time
        times.append(conflict_duration)

        # Only score if a valid result was returned
        if conflict_status == 2 or not valid_conflict_response:
            continue
        detected_conflict = (conflict_status == 1)

        # Update counts
        if detected_conflict and gt_conflict:
            TP += 1
        elif detected_conflict and not gt_conflict:
            FP += 1
        elif not detected_conflict and not gt_conflict:
            TN += 1
        elif not detected_conflict and gt_conflict:
            FN += 1

    total = TP + TN + FP + FN
    accuracy = (TP + TN) / total if total > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    fpr = FP / (FP + TN) if (FP + TN) > 0 else 0
    avg_time = sum(times) / len(times) if times else 0

    summary_rows.append([
        model, TP, FP, TN, FN, round(accuracy, 4), round(precision, 4),
        round(recall, 4), round(f1, 4), round(fpr, 4), round(avg_time, 4)
    ])

# ---- Write summary CSV ----
with open("conflict_detection_model_summaries.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Model", "TP", "FP", "TN", "FN", "Accuracy", "Precision", "Recall", "F1", "FPR", "Avg Time (s)"])
    for row in summary_rows:
        writer.writerow(row)

print("Processing completed. Model summaries saved to CSV.")